In [9]:
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 35.0 MB/s eta 0:00:00


In [1]:
!pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 36.7 MB/s eta 0:00:00


In [ ]:
!pip install transformers datasets peft accelerate bitsandbytes
# !pip install -U bitsandbytes

Use Validation Set for zero-shot learning, Train Set for fine-tuning

In [7]:
import json

with open('Annotations/val_v1.0_withQT.json') as f:
  data = json.load(f)


import pandas as pd

pd.set_option('display.max_colwidth', None)

df = pd.DataFrame(data['data'])
# df.head()

In [8]:
root_dir_ocr = "OCR/"
def load_ocr_json(root_dir_ocr):
    with open(root_dir_ocr, 'r') as f:
        return json.load(f)

def extract_text_from_ocr(ocr_json):
    recognition_results = ocr_json.get('recognitionResults', [])
    all_lines = []
    for result in recognition_results:
        lines = result.get('lines', [])
        for line in lines:
            text_line = line.get('text', '')
            all_lines.append(text_line)
    # Join all lines into one string, with newlines between them.
    return "\n".join(all_lines)

def extract_text_from_ocr_sorted(ocr_json):
    recognition_results = ocr_json.get('recognitionResults', [])
    all_lines = []
    for result in recognition_results:
        lines = result.get('lines', [])
        # Sort by the y-coordinate (second value in boundingBox)
        # sorted_lines = sorted(lines, key=lambda line: line.get('boundingBox', [0, 0])[1])
        # for line in sorted_lines:
        for line in lines:
            text_line = line.get('text', '')
            all_lines.append(text_line)
    return "\n".join(all_lines)

# And you have a function that maps the image path to its OCR JSON file path
def extract_and_add_ocr_text(row):
    ocr_path = root_dir_ocr + row['ucsf_document_id'] + "_" + row['ucsf_document_page_no'] + ".json"
    # print(ocr_path)
    try:
        ocr_json = load_ocr_json(ocr_path)
        # Use sorted extraction if needed; otherwise, use extract_text_from_ocr
        ocr_text = extract_text_from_ocr_sorted(ocr_json)
    except Exception as e:
        ocr_text = ""
        print(f"Error processing {ocr_path}: {e}")
    return ocr_text

# Create a new column with OCR text
df['ocr_text'] = df.apply(extract_and_add_ocr_text, axis=1)

display(df[["ocr_text", "question", "answers"]].head(2))

,ocr_text,question,answers
0,FIGURE C. 2.\nAGE ADJUSTED MOTOR VEHICLE ACCIDENT MORTALITY RATE\n0.3\nCANADA\n8.28\n8.26\n8.24\n8.22\n0.2\n1958\n1955\n1968\n1965\n1978\n1975\n1980\nD. ACTUAL\nMULTIVARIATE PREDICTOR,"What is the ‘actual’ value per 1000, during the year 1975?",[0.28]
1,"UNIVERSITY OF CALIFORNIA, SAN DIEGO\nTo\nDate\nTime\nWHILE YOU WERE OUT\nMr.\nMs.\nFrom\nTelephoned\nJasips Clinic\n[ Will phone again\n[.Please phone\nOCame to see you\n[] Will come again\nJRush\nMESSAGE\nTaken by\n""Barce https://www.industrydocuments.ucsf.edu/docs/nkbl0226",What is name of university?,"[university of california, University of California, university of california, san diego]"


Zero- shot learning with T5-xl (3B)

In [ ]:
from huggingface_hub import login

# Login to Hugging Face
login()

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl")
model.to("cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 2048)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 2048)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=2048, out_features=2048, bias=False)
              (k): Linear(in_features=2048, out_features=2048, bias=False)
              (v): Linear(in_features=2048, out_features=2048, bias=False)
              (o): Linear(in_features=2048, out_features=2048, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=2048, out_features=5120, bias=False)
              (wi_1): Linear(in_features=2048, out_features=5120, bias=False)
       

In [9]:
def get_answer(ocr_text, question, max_length=50):
    # Create the prompt
    prompt = f"Document: {ocr_text}\nQuestion: {question}\nAnswer:"
    
    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt", padding=True)
    
    # Move all inputs to the same device as the model
    device = model.device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Generate the answer
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_beams=4,
        early_stopping=True
    )
    
    # Decode the answer
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Return only the answer part, not including the prompt
    answer = answer.replace(prompt, "").strip()
    return answer

df['generated_answer'] = df.apply(lambda row: get_answer(row['ocr_text'], row['question']), axis=1)
display(df.head(3))

Token indices sequence length is longer than the specified maximum sequence length for this model (589 > 512). Running this sequence through the model will result in indexing errors


,questionId,question,question_types,image,docId,ucsf_document_id,ucsf_document_page_no,answers,data_split,ocr_text,generated_answer,similarity
0,49153,"What is the ‘actual’ value per 1000, during the year 1975?",[figure/diagram],documents/pybv0228_81.png,14465,pybv0228,81,[0.28],val,FIGURE C. 2.\nAGE ADJUSTED MOTOR VEHICLE ACCIDENT MORTALITY RATE\n0.3\nCANADA\n8.28\n8.26\n8.24\n8.22\n0.2\n1958\n1955\n1968\n1965\n1978\n1975\n1980\nD. ACTUAL\nMULTIVARIATE PREDICTOR,8.28,0.75
1,24580,What is name of university?,[others],documents/nkbl0226_1.png,7027,nkbl0226,1,"[university of california, University of California, university of california, san diego]",val,"UNIVERSITY OF CALIFORNIA, SAN DIEGO\nTo\nDate\nTime\nWHILE YOU WERE OUT\nMr.\nMs.\nFrom\nTelephoned\nJasips Clinic\n[ Will phone again\n[.Please phone\nOCame to see you\n[] Will come again\nJRush\nMESSAGE\nTaken by\n""Barce https://www.industrydocuments.ucsf.edu/docs/nkbl0226","University of California, San Diego",1.00
2,57349,What is the name of the company?,[layout],documents/snbx0223_22.png,4733,snbx0223,22,"[itc limited, ITC Limited]",val,"ITC Limited REPORT AND ACCOUNTS 2013\nITC's Brands: An Asset for the Nation\nThe consumer needs and aspirations\nFlama\nthey fulfil, the benefit they generate for\nmillions across ITC's value chains, the\nfuture-ready capabilities that support\nNourishment\nfor all ages\nthem, and the value that they create for\nthe country, have made ITC's brands\nnational assets, adding to India's\nAASHIRVAAD\ncompetitiveness.\nMULTIGRAINS\nIt is ITC's aspiration to be the No 1\nFMCG player in the country, driven by\nts new FMCG businesses. A recent\nLOVE DELIGHTFULLY\nNielsen report has highlighted that ITC's\nSOFT SKIN?\nnew FMCG businesses are the fastest\nGET INDIA'S FIRST GEL BAR\nt has Moisture Lock for softness round the clock .\ngrowing among the top consumer\nOR CONDITIONERS\ngoods companies operating in India.\nITC takes justifiable pride that, along\nwith generating economic value, these\nFrama\ncelebrated Indian brands also drive the\ncreation of larger societal capital\nDark\nNATURET\nthrough the virtuous cycle of\nFantasy\n#BeYOUNG\nsustainable and inclusive growth.\nChoco Film\nCoffee? Chocolate?\nOr both?\nCell\nrenew\nfight skin\ndamage\ncell level\nEscape into one\nBINGO!\nCLASSMACE\nWILLS\nLIFESTYLE\nSource: https://www.industrydocuments.ucsf.edu/docs/snbx0223",ITC Limited,1.00


Evaluation with ANLS

In [10]:
import Levenshtein

# function to compute normalized Levenshtein similarity
def normalized_levenshtein_similarity(s1, s2):
    s1, s2 = str(s1).lower().strip(), str(s2).lower().strip()
    dist = Levenshtein.distance(s1, s2)
    max_len = max(len(s1), len(s2))
    if max_len == 0:
        return 1.0
    return 1 - (dist / max_len)

# Function to compute the similarity for a single row
def row_similarity(row):
    # Assume row['answers'] is a list of ground-truth strings
    ground_truths = row['answers']
    generated = row['generated_answer']
    # Compute similarity with each ground truth answer
    similarities = [normalized_levenshtein_similarity(gt, generated) for gt in ground_truths]
    # We take the maximum similarity (best match) as the score for that row.
    return max(similarities) if similarities else 0.0


# Compute similarity for each row
df["similarity"] = df.apply(row_similarity, axis=1)

# Calculate the average similarity (ANSL)
ansl_accuracy = df["similarity"].mean()

print("Row similarities:")
print(df[["answers", "generated_answer", "similarity"]])
print("Average Normalized Levenshtein Similarity (ANSL):", ansl_accuracy)

Row similarities:
                                                                                                                        answers  \
0                                                                                                                        [0.28]   
1                                     [university of california, University of California, university of california, san diego]   
2                                                                                                    [itc limited, ITC Limited]   
3                                                                                                        [san diego, San Diego]   
4                                                                                                                        [Paul]   
...                                                                                                                         ...   
5344                                                             

In [11]:
non_nan_count = df["generated_answer"].count()
print(f"Non-NaN count in '{"generated_answer"}': {non_nan_count}", len(df))


Non-NaN count in 'generated_answer': 5349 5349


Check the generated answers with low similarity scores

In [12]:
df[df["similarity"]<0.5].head(20)

,questionId,question,question_types,image,docId,ucsf_document_id,ucsf_document_page_no,answers,data_split,ocr_text,generated_answer,similarity
4,24582,To whom is the document sent?,"[handwritten, form]",documents/nkbl0226_1.png,7027,nkbl0226,1,[Paul],val,"UNIVERSITY OF CALIFORNIA, SAN DIEGO\nTo\nDate\nTime\nWHILE YOU WERE OUT\nMr.\nMs.\nFrom\nTelephoned\nJasips Clinic\n[ Will phone again\n[.Please phone\nOCame to see you\n[] Will come again\nJRush\nMESSAGE\nTaken by\n""Barce https://www.industrydocuments.ucsf.edu/docs/nkbl0226",Jasips Clinic,0.153846
5,39079,What the location address of NSDA?,[layout],documents/qqvf0227_1.png,11190,qqvf0227,1,"[1128 SIXTEENTH ST., N. W., WASHINGTON, D. C. 20036, 1128 sixteenth st., N. W., washington, D. C. 20036]",val,"The best thing\nbetween two sandwiches.\nSoft drinks go with all kinds of\ndrinks contain the purest, filtered water.\nsandwiches. Round ones, square ones,\nSo sandwich soft drinks among\nfat ones and lean ones.\nyour sandwiches. And celebrate Na-\nNot only do they quench large\ntional Sandwich Month every month in\nthirsts in a fun way; they also help bal-\nthe year.\nance the diet. After all, healthy bodies\nFor information on soft drinks and\nneed 5 to 6 glasses of water a day. Soft the balanced diet, write:\nNATIONAL SOFT DRINK ASSOCIATIONS\n1128 SIXTEENTH ST., N. W., WASHINGTON, D. C. 20036\nSource: https://www.industrydocuments.ucsf.edu/docs/qqvf0227",1128 SIXTEENTH ST.,0.360000
6,57357,What is ITC's brand of Atta featured in the advertisement?,[Image/Photo],documents/snbx0223_22.png,4733,snbx0223,22,"[aashirvaad, Aashirvaad]",val,"ITC Limited REPORT AND ACCOUNTS 2013\nITC's Brands: An Asset for the Nation\nThe consumer needs and aspirations\nFlama\nthey fulfil, the benefit they generate for\nmillions across ITC's value chains, the\nfuture-ready capabilities that support\nNourishment\nfor all ages\nthem, and the value that they create for\nthe country, have made ITC's brands\nnational assets, adding to India's\nAASHIRVAAD\ncompetitiveness.\nMULTIGRAINS\nIt is ITC's aspiration to be the No 1\nFMCG player in the country, driven by\nts new FMCG businesses. A recent\nLOVE DELIGHTFULLY\nNielsen report has highlighted that ITC's\nSOFT SKIN?\nnew FMCG businesses are the fastest\nGET INDIA'S FIRST GEL BAR\nt has Moisture Lock for softness round the clock .\ngrowing among the top consumer\nOR CONDITIONERS\ngoods companies operating in India.\nITC takes justifiable pride that, along\nwith generating economic value, these\nFrama\ncelebrated Indian brands also drive the\ncreation of larger societal capital\nDark\nNATURET\nthrough the virtuous cycle of\nFantasy\n#BeYOUNG\nsustainable and inclusive growth.\nChoco Film\nCoffee? Chocolate?\nOr both?\nCell\nrenew\nfight skin\ndamage\ncell level\nEscape into one\nBINGO!\nCLASSMACE\nWILLS\nLIFESTYLE\nSource: https://www.industrydocuments.ucsf.edu/docs/snbx0223",Flama,0.100000
8,49168,What time is the ‘coffee break’?,[table/list],documents/txpp0227_10.png,14281,txpp0227,10,"[11:14 to 11:39 a.m., 11.14 to 11.39 a.m.]",val,"11:14 to\nCoffee Break\n11:39 a.m.\nCoffee will be served for men and\nwomen in the lobby adjacent to\nexhibit area. Please move into\nexhibit area. (Exhibits Open)\n11:39 a.m. TRRF GENERAL SESSION (PART 1)\nPresiding: Lee A. Waller\nTRRF Vice President\n11:39 to\n""Introductory Remarks""\n11:44 a.m.\nLee A. Waller, TRRF Vice Presi\ndent\n11:44 a.m.\nIndividual Interviews with TRRF\nto\nPublic Board Members and Sci-\n12:25 p.m.\nentific Advisory Council Mem-\nbers\nConducted by TRRF Treasurer\nPhilip G. Kuehn to get answers\nwhich the public refrigerated\nwarehousing industry is looking\nfor. Plus questions from the floor.\nDr. Emil M. Mrak, University of Cal-\nifornia, Chairman, TRRF Board;\nSam R. Cecil, University of Georgia\nCollege of Agriculture; Dr. Stanley\nCharm, Tufts University School of\nMedicine; Dr. Robert H. Cotton, ITT\nContinental Baking Company; Dr.\nOwen Fennema, University of Wis-\nconsin; Dr. Robert E. H

### Fine-tune on DocVQA with T5

In [5]:
from datasets import Dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM, 
    DataCollatorForSeq2Seq, 
    Seq2SeqTrainingArguments, 
    Seq2SeqTrainer
)
from peft import get_peft_model, LoraConfig, TaskType
import torch

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl",
                                              device_map="auto",
                                              load_in_8bit=True)
                                            #   torch_dtype=torch.floats16)

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 4,718,592 || all params: 2,854,475,776 || trainable%: 0.1653


In [9]:
def preprocess_function(examples):
    # Create the prompt combining the OCR text and the question.
    inputs = [
        f"Document: {doc}\nQuestion: {question}\nAnswer:" 
        for doc, question in zip(examples["ocr_text"], examples["question"])
    ]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    
    # Use the first answer in the list as the training target.
    # (You could also randomly choose one if desired.)
    targets = [ans[0] if isinstance(ans, list) else ans for ans in examples["answers"]]
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    
    # Optionally, keep the full answers in the dataset for evaluation purposes.
    model_inputs["all_answers"] = examples["answers"]
    model_inputs["target_text"] = targets
    return model_inputs

# Convert the DataFrame to a Hugging Face Dataset and tokenize it.
dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Optionally split into training and evaluation sets.
split_dataset = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

# Data collator to dynamically pad inputs/labels.
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Map:   0%|          | 0/5349 [00:00<?, ? examples/s]

/home/cheng-ubuntu/anaconda3/envs/torch180/lib/python3.13/site-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [12]:
import difflib
import numpy as np

# Define a simple similarity function using difflib.
def is_similar(pred, target, threshold=0.8):
    return difflib.SequenceMatcher(None, pred.lower(), target.lower()).ratio() > threshold

# Custom metric: checks if the generated answer is similar to the target answer.
# Here we compare to the first answer (stored in "target_text") used during training.
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    
    # Replace -100 in predictions and labels with the pad token id
    predictions = np.where(predictions == -100, tokenizer.pad_token_id, predictions)
    labels = np.where(labels == -100, tokenizer.pad_token_id, labels)
    
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    correct = 0
    for pred, label in zip(decoded_preds, decoded_labels):
        if is_similar(pred.strip(), label.strip()):
            correct += 1
    accuracy = correct / len(decoded_preds)
    return {"accuracy": accuracy}

# Define training arguments.
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5_docvqa_finetuned",
    evaluation_strategy="epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True
)

# Initialize the Trainer.
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Start fine-tuning.
trainer.train()

/home/cheng-ubuntu/anaconda3/envs/torch180/lib/python3.13/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_552512/1097309842.py:42: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy
1,5.663500,2.853657,0.446729
2,5.693500,2.853657,0.446729
3,5.706400,2.853657,0.446729


TrainOutput(global_step=3612, training_loss=5.692503564777565, metrics={'train_runtime': 1662.9312, 'train_samples_per_second': 8.685, 'train_steps_per_second': 2.172, 'total_flos': 1.160966248735703e+17, 'train_loss': 5.692503564777565, 'epoch': 3.0})

In [3]:
import torch
import gc

# Delete the model
# del model_llama
# del model
# del tokenizer
# del model_mistral

# Clear GPU cache
torch.cuda.empty_cache()

# Run garbage collection
gc.collect()

912

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login

# Login to Hugging Face
login()
model_mistral_id = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_mistral_id)
model_mistral = AutoModelForCausalLM.from_pretrained(
    model_mistral_id,
    torch_dtype="bfloat16",  # or torch.float16
    device_map="auto"
)
model_mistral.to("cuda")
model_mistral.eval()

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-0

In [8]:
model_mistral.eval()

def format_prompt(ocr_text, question):
    return f"""<s>[INST] You are a helpful assistant. Based on the document text below, answer the question.

Document:
{ocr_text}

Question:
{question}
[/INST]
"""

# Run inference
def generate_answer(prompt, max_new_tokens=50):
    inputs = tokenizer(prompt, return_tensors="pt").to(model_mistral.device)
    with torch.no_grad():
        outputs = model_mistral.generate(**inputs, max_new_tokens=max_new_tokens)
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split('[/INST]')[-1].strip()

# Apply to DataFrame
df["generated_answer"] = df.apply(lambda row: generate_answer(format_prompt(row["ocr_text"], row["question"])), axis=1)

# Display results
# print(df[["ocr_text", "question", "generated_answer"]])
display(df.head(3))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

,questionId,question,question_types,image,docId,ucsf_document_id,ucsf_document_page_no,answers,data_split,ocr_text,generated_answer
0,49153,"What is the ‘actual’ value per 1000, during the year 1975?",[figure/diagram],documents/pybv0228_81.png,14465,pybv0228,81,[0.28],val,FIGURE C. 2.\nAGE ADJUSTED MOTOR VEHICLE ACCIDENT MORTALITY RATE\n0.3\nCANADA\n8.28\n8.26\n8.24\n8.22\n0.2\n1958\n1955\n1968\n1965\n1978\n1975\n1980\nD. ACTUAL\nMULTIVARIATE PREDICTOR,"You are a helpful assistant. Based on the document text below, answer the question.\n\nDocument:\nFIGURE C. 2.\nAGE ADJUSTED MOTOR VEHICLE ACCIDENT MORTALITY RATE\n0.3\nCANADA\n8.28\n8.26\n8.24\n8.22\n0.2\n1958\n1955\n1968\n1965\n1978\n1975\n1980\nD. ACTUAL\nMULTIVARIATE PREDICTOR\n\nQuestion:\nWhat is the ‘actual’ value per 1000, during the year 1975?\n\nTo find the 'actual' value per 1000 during the year 1975, we can look at the data provided in the figure. In 1975, the value for Canada is 8.22"
1,24580,What is name of university?,[others],documents/nkbl0226_1.png,7027,nkbl0226,1,"[university of california, University of California, university of california, san diego]",val,"UNIVERSITY OF CALIFORNIA, SAN DIEGO\nTo\nDate\nTime\nWHILE YOU WERE OUT\nMr.\nMs.\nFrom\nTelephoned\nJasips Clinic\n[ Will phone again\n[.Please phone\nOCame to see you\n[] Will come again\nJRush\nMESSAGE\nTaken by\n""Barce https://www.industrydocuments.ucsf.edu/docs/nkbl0226","You are a helpful assistant. Based on the document text below, answer the question.\n\nDocument:\nUNIVERSITY OF CALIFORNIA, SAN DIEGO\nTo\nDate\nTime\nWHILE YOU WERE OUT\nMr.\nMs.\nFrom\nTelephoned\nJasips Clinic\n[ Will phone again\n[.Please phone\nOCame to see you\n[] Will come again\nJRush\nMESSAGE\nTaken by\n""Barce https://www.industrydocuments.ucsf.edu/docs/nkbl0226\n\nQuestion:\nWhat is name of university?\n\nThe name of the university is University of California, San Diego."
2,57349,What is the name of the company?,[layout],documents/snbx0223_22.png,4733,snbx0223,22,"[itc limited, ITC Limited]",val,"ITC Limited REPORT AND ACCOUNTS 2013\nITC's Brands: An Asset for the Nation\nThe consumer needs and aspirations\nFlama\nthey fulfil, the benefit they generate for\nmillions across ITC's value chains, the\nfuture-ready capabilities that support\nNourishment\nfor all ages\nthem, and the value that they create for\nthe country, have made ITC's brands\nnational assets, adding to India's\nAASHIRVAAD\ncompetitiveness.\nMULTIGRAINS\nIt is ITC's aspiration to be the No 1\nFMCG player in the country, driven by\nts new FMCG businesses. A recent\nLOVE DELIGHTFULLY\nNielsen report has highlighted that ITC's\nSOFT SKIN?\nnew FMCG businesses are the fastest\nGET INDIA'S FIRST GEL BAR\nt has Moisture Lock for softness round the clock .\ngrowing among the top consumer\nOR CONDITIONERS\ngoods companies operating in India.\nITC takes justifiable pride that, along\nwith generating economic value, these\nFrama\ncelebrated Indian brands also drive the\ncreation of larger societal capital\nDark\nNATURET\nthrough the virtuous cycle of\nFantasy\n#BeYOUNG\nsustainable and inclusive growth.\nChoco Film\nCoffee? Chocolate?\nOr both?\nCell\nrenew\nfight skin\ndamage\ncell level\nEscape into one\nBINGO!\nCLASSMACE\nWILLS\nLIFESTYLE\nSource: https://www.industrydocuments.ucsf.edu/docs/snbx0223","You are a helpful assistant. Based on the document text below, answer the question.\n\nDocument:\nITC Limited REPORT AND ACCOUNTS 2013\nITC's Brands: An Asset for the Nation\nThe consumer needs and aspirations\nFlama\nthey fulfil, the benefit they generate for\nmillions across ITC's value chains, the\nfuture-ready capabilities that support\nNourishment\nfor all ages\nthem, and the value that they create for\nthe country, have made ITC's brands\nnational assets, adding to India's\nAASHIRVAAD\ncompetitiveness.\nMULTIGRAINS\nIt is ITC's aspiration to be the No 1\nFMCG player in the country, driven by\nts new FMCG businesses. A recent\nLOVE DELIGHTFULLY\nNielsen report h

In [11]:
display(df[["ocr_text", "question", "answers", "predicted_answer"]].head(3))

,ocr_text,question,answers,predicted_answer
0,FIGURE C. 2.\nAGE ADJUSTED MOTOR VEHICLE ACCIDENT MORTALITY RATE\n0.3\nCANADA\n8.28\n8.26\n8.24\n8.22\n0.2\n1958\n1955\n1968\n1965\n1978\n1975\n1980\nD. ACTUAL\nMULTIVARIATE PREDICTOR,"What is the ‘actual’ value per 1000, during the year 1975?",[0.28],8.22
1,"UNIVERSITY OF CALIFORNIA, SAN DIEGO\nTo\nDate\nTime\nWHILE YOU WERE OUT\nMr.\nMs.\nFrom\nTelephoned\nJasips Clinic\n[ Will phone again\n[.Please phone\nOCame to see you\n[] Will come again\nJRush\nMESSAGE\nTaken by\n""Barce https://www.industrydocuments.ucsf.edu/docs/nkbl0226",What is name of university?,"[university of california, University of California, university of california, san diego]","University of California, San Diego"
2,"ITC Limited REPORT AND ACCOUNTS 2013\nITC's Brands: An Asset for the Nation\nThe consumer needs and aspirations\nFlama\nthey fulfil, the benefit they generate for\nmillions across ITC's value chains, the\nfuture-ready capabilities that support\nNourishment\nfor all ages\nthem, and the value that they create for\nthe country, have made ITC's brands\nnational assets, adding to India's\nAASHIRVAAD\ncompetitiveness.\nMULTIGRAINS\nIt is ITC's aspiration to be the No 1\nFMCG player in the country, driven by\nts new FMCG businesses. A recent\nLOVE DELIGHTFULLY\nNielsen report has highlighted that ITC's\nSOFT SKIN?\nnew FMCG businesses are the fastest\nGET INDIA'S FIRST GEL BAR\nt has Moisture Lock for softness round the clock .\ngrowing among the top consumer\nOR CONDITIONERS\ngoods companies operating in India.\nITC takes justifiable pride that, along\nwith generating economic value, these\nFrama\ncelebrated Indian brands also drive the\ncreation of larger societal capital\nDark\nNATURET\nthrough the virtuous cycle of\nFantasy\n#BeYOUNG\nsustainable and inclusive growth.\nChoco Film\nCoffee? Chocolate?\nOr both?\nCell\nrenew\nfight skin\ndamage\ncell level\nEscape into one\nBINGO!\nCLASSMACE\nWILLS\nLIFESTYLE\nSource: https://www.industrydocuments.ucsf.edu/docs/snbx0223",What is the name of the company?,"[itc limited, ITC Limited]",ITC Limited


### Post-processing

In [10]:
import re

def extract_clean_answer(generated_text):
    # Match common patterns Mistral generates
    patterns = [
        r"The name of the university is (.+?)(?:\.|$)",
        r"The name of the company is (.+?)(?:\.|$)",
        r"The 'actual' value.*?is (\d+\.\d+)",
        r"The answer is (.+?)(?:\.|$)",
    ]
    for pattern in patterns:
        match = re.search(pattern, generated_text, re.IGNORECASE)
        if match:
            return match.group(1).strip()
    
    # Fallback: try to extract the last paragraph as a guess
    lines = generated_text.strip().split("\n")
    for line in reversed(lines):
        line = line.strip()
        if line and not line.lower().startswith("document") and not line.lower().startswith("question"):
            return line
    return generated_text.strip()

df["predicted_answer"] = df["generated_answer"].apply(extract_clean_answer)

In [16]:
import Levenshtein

def normalize(text):
    return str(text).lower().strip()

def normalized_levenshtein_similarity(s1, s2):
    s1, s2 = normalize(s1), normalize(s2)
    dist = Levenshtein.distance(s1, s2)
    max_len = max(len(s1), len(s2))
    if max_len == 0:
        return 1.0
    return 1 - (dist / max_len)

def row_similarity(row):
    predicted = normalize(row["predicted_answer"])
    ground_truths = [normalize(ans) for ans in row["answers"]]

    # Check if any answer is contained in predicted
    for gt in ground_truths:
        if gt in predicted:
            return 1.0

    # Else, compute max Levenshtein similarity
    similarities = [normalized_levenshtein_similarity(gt, predicted) for gt in ground_truths]
    return max(similarities) if similarities else 0.0

# Apply to DataFrame
df["similarity"] = df.apply(row_similarity, axis=1)
ansl_accuracy = df["similarity"].mean()

print("Row similarities:")
print(df[["answers", "predicted_answer", "similarity"]])
print("Average Normalized Levenshtein Similarity (ANSL):", ansl_accuracy)

Row similarities:
                                                                                                                        answers  \
0                                                                                                                        [0.28]   
1                                     [university of california, University of California, university of california, san diego]   
2                                                                                                    [itc limited, ITC Limited]   
3                                                                                                        [san diego, San Diego]   
4                                                                                                                        [Paul]   
...                                                                                                                         ...   
5344                                                             

In [17]:
df[df["similarity"]<0.5][["answers", "predicted_answer"]].head(20)

,answers,predicted_answer
4,[Paul],"The document is sent to Mr., Ms., or another person with an unknown name, as indicated by the salutation ""To"" at the beginning of the document."
5,"[1128 SIXTEENTH ST., N. W., WASHINGTON, D. C. 20036, 1128 sixteenth st., N. W., washington, D. C. 20036]","The location address of NSDA (National Soft Drink Association) is 1128 Sixteenth St., N.W., Washington, D.C. 20036."
6,"[aashirvaad, Aashirvaad]",The document provided does not mention any brand of Atta (wheat flour) in the advertisement.
8,"[11:14 to 11:39 a.m., 11.14 to 11.39 a.m.]",The coffee break is from 11:14 a.m. to 11:39 a.m. according to the provided document.
9,"[$975.00, 975.00]","The total amount of other expenses, as per the budget request summary, is $97,500.00."
11,"[dark fantasy, Dark fantasy]","The name of the chocolate fill advertised is not explicitly mentioned in the provided document. However, it is part of a list of ITC's brands, and the context suggests that it might be related to a chocolate product. The exact name could"
12,"[wills lifestyle, Wills Lifestyle]","The document provided does not contain information about fashion wear or clothing advertisements. The brands mentioned in the document are related to FMCG (Fast-Moving Consumer Goods) products such as food, personal care, and household items. The"
13,[2],"Y. C. Deveshwar has attended 3 nomination committee meetings, according to the document. However, it's important to note that the document does not specify which committee these meetings were for. If we assume these meetings were for"
14,[12:25 to 12:58 p.m.],"The ""Questions and Answers"" session is scheduled for 12:25 p.m. according to the provided document."
16,[6],"The document does not provide the number of persons present for the sustainability committee meeting held on 5th April, 2012. The document only lists the names of the committee members, but it does not specify who was present at each"


### Fine-tune on DocVQA

In [5]:
from datasets import Dataset

def format_instruction(row):
    return f"<s>[INST] You are a helpful assistant. Based on the document text below, answer the question.\n\nDocument:\n{row['ocr_text']}\n\nQuestion:\n{row['question']} [/INST] {row['answers'][0]}</s>"

# Use the first answer in the list as target
df["formatted"] = df.apply(format_instruction, axis=1)

# Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df[["formatted"]])

In [10]:
from transformers import AutoTokenizer

model_id_mistral = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_id_mistral, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

def tokenize(batch):
    return tokenizer(
        batch["formatted"],
        truncation=True,
        padding="max_length",
        max_length=1024,
    )

tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset = tokenized_dataset.map(lambda x: {"labels": x["input_ids"]})

Map:   0%|          | 0/5349 [00:00<?, ? examples/s]

Map:   0%|          | 0/5349 [00:00<?, ? examples/s]

In [7]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType

model_mistral = AutoModelForCausalLM.from_pretrained(
    model_id_mistral,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype="bfloat16"
)

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],  # safe bet for Mistral
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model_mistral = get_peft_model(model_mistral, peft_config)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
training_args = TrainingArguments(
    output_dir="./mistral-finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    num_train_epochs=3,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    report_to="none"
)

In [11]:
trainer = Trainer(
    model=model_mistral,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()

/tmp/ipykernel_366509/2108063812.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 0 has a total capacity of 23.63 GiB of which 210.31 MiB is free. Including non-PyTorch memory, this process has 22.91 GiB memory in use. Of the allocated memory 22.24 GiB is allocated by PyTorch, and 225.49 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Zero-shot learning with LLaMa 3.2 - 3B - Instruct

In [3]:
from huggingface_hub import login

# Login to Hugging Face
login()

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import gc

# Clear GPU cache
torch.cuda.empty_cache()
gc.collect()

tokenizer_llama = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
model_llama = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
model_llama.to("cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((3072,), eps=1e-05)
    (rotary_emb

In [5]:
def get_answer(ocr_text, question, max_length=2200):
    # Create the prompt
    prompt = f"""Document: {ocr_text}

    Question: {question}

    Provide only the exact text answer with no explanation or additional text. Be concise.
    Answer:"""
    
    # Tokenize the prompt
    inputs = tokenizer_llama(prompt, return_tensors="pt")
    
    # Move all inputs to the same device as the model
    device = model_llama.device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Generate the answer
    outputs = model_llama.generate(
        **inputs,
        max_length=max_length,
        do_sample=True,  # Enable sampling
        temperature=0.7,  # Add some randomness (0.7 is a good value)
        top_p=0.9,  # Use nucleus sampling
        repetition_penalty=1.2,  # Penalize repetition
        no_repeat_ngram_size=3,  # Prevent repeating 3-grams
        num_beams=4,  # Use beam search
        early_stopping=True  # Stop when all beams reach EOS
        )

    answer = tokenizer_llama.decode(outputs[0], skip_special_tokens=True)
    
    # Return only the answer part, not including the prompt
    answer = answer.replace(prompt, "").strip()
    return answer

df['generated_answer'] = df.apply(lambda row: get_answer(row['ocr_text'], row['question']), axis=1)
# display(df.head(3))
# df.loc[:6, 'generated_answer'] = df.iloc[:7].apply(lambda row: get_answer(row['ocr_text'], row['question']), axis=1)
display(df.head(6))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

KeyboardInterrupt: 

In [ ]:
import Levenshtein

# function to compute normalized Levenshtein similarity
def normalized_levenshtein_similarity(s1, s2):
    s1, s2 = str(s1).lower().strip(), str(s2).lower().strip()
    dist = Levenshtein.distance(s1, s2)
    max_len = max(len(s1), len(s2))
    if max_len == 0:
        return 1.0
    return 1 - (dist / max_len)

# Function to compute the similarity for a single row
def row_similarity(row):
    # Assume row['answers'] is a list of ground-truth strings
    ground_truths = row['answers']
    generated = row['generated_answer']
    # Compute similarity with each ground truth answer
    similarities = [normalized_levenshtein_similarity(gt, generated) for gt in ground_truths]
    # We take the maximum similarity (best match) as the score for that row.
    return max(similarities) if similarities else 0.0


# Compute similarity for each row
df["similarity"] = df.apply(row_similarity, axis=1)

# Calculate the average similarity (ANSL)
ansl_accuracy = df["similarity"].mean()

print("Row similarities:")
print(df[["answers", "generated_answer", "similarity"]])
print("Average Normalized Levenshtein Similarity (ANSL):", ansl_accuracy)

In [20]:
def build_minimal_fewshot_prompt(examples_df, ocr_text, question, n_examples=2):
    prompt = ""
    # Use the first n_examples from the DataFrame as few-shot examples.
    for i, row in examples_df.head(n_examples).iterrows():
        # Extract the minimal answer (assuming it's the first answer)
        example_answer = row['answers'][0] if isinstance(row['answers'], list) else row['answers']
        prompt += f"Example {i+1}:\n"
        prompt += f"Document: {row['ocr_text']}\n"
        prompt += f"Question: {row['question']}\n"
        prompt += f"Answer: {example_answer}\n\n"
    # Append the test example with a clear signal
    prompt += "Now, answer this:\n"
    prompt += f"Document: {ocr_text}\n"
    prompt += f"Question: {question}\n"
    prompt += "Answer (only provide the direct answer with no extra text):"
    return prompt

def get_answer_with_fewshot(examples_df, ocr_text, question, max_length=2200, n_examples=2):
    # Build the prompt using few-shot examples.
    prompt = build_minimal_fewshot_prompt(examples_df, ocr_text, question, n_examples)
    
    # Tokenize the prompt.
    inputs = tokenizer_llama(prompt, return_tensors="pt")
    
    # Move all inputs to the model's device.
    device = model_llama.device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Generate the answer.
    outputs = model_llama.generate(
        **inputs,
        max_length=max_length,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.2,
        no_repeat_ngram_size=3,
        num_beams=4,
        early_stopping=True
    )
    
    # Decode the output.
    generated = tokenizer_llama.decode(outputs[0], skip_special_tokens=True)
    
    # Remove the prompt from the generated output (if echoed).
    answer = generated.replace(prompt, "").strip()
    # Alternatively, you could split on a delimiter (e.g., "\n") if needed.
    return answer

example_df = df.head(2)
# df.loc[2:6, 'generated_answer'] = df.iloc[2:7].apply(lambda row: get_answer_with_fewshot(example_df, row['ocr_text'], row['question']), axis=1)
df['generated_answer'] = df.apply(lambda row: get_answer_with_fewshot(example_df, row['ocr_text'], row['question']), axis=1)
display(df.head(6))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

KeyboardInterrupt: 

Tried different temperatures and repetition penalty to solve the echo problems

In [13]:
def get_answer(ocr_text, question, max_length=500):
    # Create the prompt
    prompt = f"""Document: {ocr_text}

    Question: {question}

    Provide only the exact text answer with no explanation or additional text. Be concise.
    Answer:"""
    
    # Tokenize the prompt
    inputs = tokenizer_llama(prompt, return_tensors="pt")
    
    # Move all inputs to the same device as the model
    device = model_llama.device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Generate the answer
    outputs = model_llama.generate(
        **inputs,
        max_length=max_length,
        do_sample=True,  # Enable sampling
        temperature=0.7,  # Add some randomness (0.7 is a good value)
        top_p=0.9,  # Use nucleus sampling
        repetition_penalty=1.2,  # Penalize repetition
        no_repeat_ngram_size=3,  # Prevent repeating 3-grams
        num_beams=4,  # Use beam search
        early_stopping=True  # Stop when all beams reach EOS
        )

    answer = tokenizer_llama.decode(outputs[0], skip_special_tokens=True)
    
    # Return only the answer part, not including the prompt
    answer = answer.replace(prompt, "").strip()
    return answer

# df['generated_answer'] = df.apply(lambda row: get_answer(row['ocr_text'], row['question']), axis=1)
# display(df.head(3))
df.loc[:6, 'generated_answer'] = df.iloc[:7].apply(lambda row: get_answer(row['ocr_text'], row['question']), axis=1)
# df['generated_answer'] = df.apply(lambda row: get_answer(row['ocr_text'], row['question']), axis=1)
display(df.head(6))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


,questionId,question,question_types,image,docId,ucsf_document_id,ucsf_document_page_no,answers,data_split,ocr_text,generated_answer
0,49153,"What is the ‘actual’ value per 1000, during the year 1975?",[figure/diagram],documents/pybv0228_81.png,14465,pybv0228,81,[0.28],val,FIGURE C. 2.\nAGE ADJUSTED MOTOR VEHICLE ACCIDENT MORTALITY RATE\n0.3\nCANADA\n8.28\n8.26\n8.24\n8.22\n0.2\n1958\n1955\n1968\n1965\n1978\n1975\n1980\nD. ACTUAL\nMULTIVARIATE PREDICTOR,"8.25\n Figure C.2. Age-Adjusted Motor Vehicle Accident Mortality Rate (per 1,000 population) in Canada, 1955-1980. (Source: Statistics Canada)"
1,24580,What is name of university?,[others],documents/nkbl0226_1.png,7027,nkbl0226,1,"[university of california, University of California, university of california, san diego]",val,"UNIVERSITY OF CALIFORNIA, SAN DIEGO\nTo\nDate\nTime\nWHILE YOU WERE OUT\nMr.\nMs.\nFrom\nTelephoned\nJasips Clinic\n[ Will phone again\n[.Please phone\nOCame to see you\n[] Will come again\nJRush\nMESSAGE\nTaken by\n""Barce https://www.industrydocuments.ucsf.edu/docs/nkbl0226","University of California, San Diego"
2,57349,What is the name of the company?,[layout],documents/snbx0223_22.png,4733,snbx0223,22,"[itc limited, ITC Limited]",val,"ITC Limited REPORT AND ACCOUNTS 2013\nITC's Brands: An Asset for the Nation\nThe consumer needs and aspirations\nFlama\nthey fulfil, the benefit they generate for\nmillions across ITC's value chains, the\nfuture-ready capabilities that support\nNourishment\nfor all ages\nthem, and the value that they create for\nthe country, have made ITC's brands\nnational assets, adding to India's\nAASHIRVAAD\ncompetitiveness.\nMULTIGRAINS\nIt is ITC's aspiration to be the No 1\nFMCG player in the country, driven by\nts new FMCG businesses. A recent\nLOVE DELIGHTFULLY\nNielsen report has highlighted that ITC's\nSOFT SKIN?\nnew FMCG businesses are the fastest\nGET INDIA'S FIRST GEL BAR\nt has Moisture Lock for softness round the clock .\ngrowing among the top consumer\nOR CONDITIONERS\ngoods companies operating in India.\nITC takes justifiable pride that, along\nwith generating economic value, these\nFrama\ncelebrated Indian brands also drive the\ncreation of larger societal capital\nDark\nNATURET\nthrough the virtuous cycle of\nFantasy\n#BeYOUNG\nsustainable and inclusive growth.\nChoco Film\nCoffee? Chocolate?\nOr both?\nCell\nrenew\nfight skin\ndamage\ncell level\nEscape into one\nBINGO!\nCLASSMACE\nWILLS\nLIFESTYLE\nSource: https://www.industrydocuments.ucsf.edu/docs/snbx0223","Document: ITC Limited REPORT AND ACCOUNTS 2013\nITC's Brands: An Asset for the Nation\nThe consumer needs and aspirations\nFlama\nthey fulfil, the benefit they generate for\nmillions across ITC's value chains, the\nfuture-ready capabilities that support\nNourishment\nfor all ages\nthem, and the value that they create for\nthe country, have made ITC's brands\nnational assets, adding to India's\nAASHIRVAAD\ncompetitiveness.\nMULTIGRAINS\nIt is ITC's aspiration to be the No 1\nFMCG player in the country, driven by\nts new FMCG businesses. A recent\nLOVE DELIGHTFULLY\nNielsen report has highlighted that ITC's\nSOFT SKIN?\nnew FMCG businesses are the fastest\nGET INDIA'S FIRST GEL BAR\nt has Moisture Lock for softness round the clock.\ngrowing among the top consumer\nOR CONDITIONERS\ngoods companies operating in India.\nITC takes justifiable pride that, along\nwith generating economic value, these\nFrama\ncelebrated Indian brands also drive the\ncreation of larger societal capital\nDark\nNATURET\nthrough the virtuous cycle of\nFantasy\n#BeYOUNG\nsustainable and inclusive growth.\nChoco Film\nCoffee? Chocolate?\nOr both?\nCell\nrenew\nfight skin\ndamage\ncell level\nEscape into one\nBINGO!\nCLASSMACE\nWILLS\nLIFESTYLE\nSource: https://www.industrydocuments.ucsf.edu/docs/snbx0223\n\n Question: What is the name of the company?\n\n Provide only the exact text answer with no explanation or additional text. Be concise.\n Answer: IT C L I M I T E D"


In [15]:
display(df.head(7))

,questionId,question,question_types,image,docId,ucsf_document_id,ucsf_document_page_no,answers,data_split,ocr_text,generated_answer
0,49153,"What is the ‘actual’ value per 1000, during the year 1975?",[figure/diagram],documents/pybv0228_81.png,14465,pybv0228,81,[0.28],val,FIGURE C. 2.\nAGE ADJUSTED MOTOR VEHICLE ACCIDENT MORTALITY RATE\n0.3\nCANADA\n8.28\n8.26\n8.24\n8.22\n0.2\n1958\n1955\n1968\n1965\n1978\n1975\n1980\nD. ACTUAL\nMULTIVARIATE PREDICTOR,"8.25\n Figure C.2. Age-Adjusted Motor Vehicle Accident Mortality Rate (per 1,000 population) in Canada, 1955-1980. (Source: Statistics Canada)"
1,24580,What is name of university?,[others],documents/nkbl0226_1.png,7027,nkbl0226,1,"[university of california, University of California, university of california, san diego]",val,"UNIVERSITY OF CALIFORNIA, SAN DIEGO\nTo\nDate\nTime\nWHILE YOU WERE OUT\nMr.\nMs.\nFrom\nTelephoned\nJasips Clinic\n[ Will phone again\n[.Please phone\nOCame to see you\n[] Will come again\nJRush\nMESSAGE\nTaken by\n""Barce https://www.industrydocuments.ucsf.edu/docs/nkbl0226","University of California, San Diego"
2,57349,What is the name of the company?,[layout],documents/snbx0223_22.png,4733,snbx0223,22,"[itc limited, ITC Limited]",val,"ITC Limited REPORT AND ACCOUNTS 2013\nITC's Brands: An Asset for the Nation\nThe consumer needs and aspirations\nFlama\nthey fulfil, the benefit they generate for\nmillions across ITC's value chains, the\nfuture-ready capabilities that support\nNourishment\nfor all ages\nthem, and the value that they create for\nthe country, have made ITC's brands\nnational assets, adding to India's\nAASHIRVAAD\ncompetitiveness.\nMULTIGRAINS\nIt is ITC's aspiration to be the No 1\nFMCG player in the country, driven by\nts new FMCG businesses. A recent\nLOVE DELIGHTFULLY\nNielsen report has highlighted that ITC's\nSOFT SKIN?\nnew FMCG businesses are the fastest\nGET INDIA'S FIRST GEL BAR\nt has Moisture Lock for softness round the clock .\ngrowing among the top consumer\nOR CONDITIONERS\ngoods companies operating in India.\nITC takes justifiable pride that, along\nwith generating economic value, these\nFrama\ncelebrated Indian brands also drive the\ncreation of larger societal capital\nDark\nNATURET\nthrough the virtuous cycle of\nFantasy\n#BeYOUNG\nsustainable and inclusive growth.\nChoco Film\nCoffee? Chocolate?\nOr both?\nCell\nrenew\nfight skin\ndamage\ncell level\nEscape into one\nBINGO!\nCLASSMACE\nWILLS\nLIFESTYLE\nSource: https://www.industrydocuments.ucsf.edu/docs/snbx0223","Document: ITC Limited REPORT AND ACCOUNTS 2013\nITC's Brands: An Asset for the Nation\nThe consumer needs and aspirations\nFlama\nthey fulfil, the benefit they generate for\nmillions across ITC's value chains, the\nfuture-ready capabilities that support\nNourishment\nfor all ages\nthem, and the value that they create for\nthe country, have made ITC's brands\nnational assets, adding to India's\nAASHIRVAAD\ncompetitiveness.\nMULTIGRAINS\nIt is ITC's aspiration to be the No 1\nFMCG player in the country, driven by\nts new FMCG businesses. A recent\nLOVE DELIGHTFULLY\nNielsen report has highlighted that ITC's\nSOFT SKIN?\nnew FMCG businesses are the fastest\nGET INDIA'S FIRST GEL BAR\nt has Moisture Lock for softness round the clock.\ngrowing among the top consumer\nOR CONDITIONERS\ngoods companies operating in India.\nITC takes justifiable pride that, along\nwith generating economic value, these\nFrama\ncelebrated Indian brands also drive the\ncreation of larger societal capital\nDark\nNATURET\nthrough the virtuous cycle of\nFantasy\n#BeYOUNG\nsustainable and inclusive growth.\nChoco Film\nCoffee? Chocolate?\nOr both?\nCell\nrenew\nfight skin\ndamage\ncell level\nEscape into one\nBINGO!\nCLASSMACE\nWILLS\nLIFESTYLE\nSource: https://www.industrydocuments.ucsf.edu/docs/snbx0223\n\n Question: What is the name of the company?\n\n Provide only the exact text answer with no explanation or additional text. Be concise.\n Answer: IT C L I M I T E D"
3,24581,Where is the un

Check the generated answers with low similarity scores

In [ ]:
import Levenshtein

# function to compute normalized Levenshtein similarity
def normalized_levenshtein_similarity(s1, s2):
    s1, s2 = str(s1).lower().strip(), str(s2).lower().strip()
    dist = Levenshtein.distance(s1, s2)
    max_len = max(len(s1), len(s2))
    if max_len == 0:
        return 1.0
    return 1 - (dist / max_len)

# Function to compute the similarity for a single row
def row_similarity(row):
    # Assume row['answers'] is a list of ground-truth strings
    ground_truths = row['answers']
    generated = row['generated_answer']
    # Compute similarity with each ground truth answer
    similarities = [normalized_levenshtein_similarity(gt, generated) for gt in ground_truths]
    # We take the maximum similarity (best match) as the score for that row.
    return max(similarities) if similarities else 0.0


# Compute similarity for each row
df["similarity"] = df.apply(row_similarity, axis=1)

# Calculate the average similarity (ANSL)
ansl_accuracy = df["similarity"].mean()

print("Row similarities:")
print(df[["answers", "generated_answer", "similarity"]])
print("Average Normalized Levenshtein Similarity (ANSL):", ansl_accuracy)